# <center>Laboratorium Analiza i bazy danych </center>

## <center>Łańcuchy znaków i wyrażenia regularne</center>

## Wprowadzenie

W każdym silniku baz danych istnieją mechanizmy do porównywania, dopasowywania i manipulowania ciągami znaków (*string*). 

Oprócz podstawowej funkcjonalności polegającej na odpowiedzi na pytanie „czy ten ciąg pasuje do tego wzoru?” w bazach danych istnieją operatory i funkcje do wyodrębniania, zastępowania i podziałów pasujących podciągów do wzorca podanego przez użytkownika.


Istnieją trzy osobne podejścia do dopasowywania wzorców zapewniane przez PostgreSQL:  
- LIKE/ILIKE, 
- SIMILAR TO (standard SQL:1999),  
- wyrażenia regularne w stylu POSIX. 

Ten dokument ma na celu przybliżenie podstawowych funkcji działania na ciągach znakowych w PostgreSQL.

## LIKE/ILIKE

Wyrażenie LIKE zwraca wartość *true*, jeśli ciąg znaków odpowiada dokładnie podanemu wzorcowi. ILIKE natomiast to klauzula umożliwiająca dopasowanie wzorca jednak nie zwraca ona uwagę na wielkość liter. Wyrażeniem przeciwnym jest NOT LIKE lub NOT ILIKE. Składnia tego wyrażenia to:

string (LIKE|ILIKE) wzorzec,
string NOT (LIKE|ILIKE) wzorzec

Przy tym typie klauzuli wzorce tworzymy przy użyciu dwóch operatorów:
- _ - zastępuje pojedynczy znak,
- % - zastępuje dowolną długość znaków.

#### Przykład:
|Wyrażenie|Wynik|
|---|---:|
|'abc' LIKE 'abc'|   true|  
|'abc' LIKE 'a%'|    true|  
|'abc' LIKE '_b_'|   true|  
|'abc' LIKE 'c'   |   false|  
|'ABC' ILIKE 'abc'|   true | 
|'ABC' LIKE 'abc'|    false| 

Klauzule LIKE/ILIKE można zastąpić operatorami:
- ~~ równoważny do LIKE
- ~~\*  równoważny do ILIKE  
- !~~  równoważny do LIKE  
- !~~\* równoważny do NOT ILIKE

## SIMILAR TO
Klauzula SIMILAR TO tak samo jak LIKE/ILIKE zwraca wartość *true* lub *false* w zależności od tego, czy podany wzorzec pasuje do podanego ciągu. Różnica pomiędzy tymi operatorami polega na tym, że SIMILAR TO interpretuje wzorzec za pomocą definicji wyrażenia regularnego w standardzie SQL. Wyrażenia regularne SQL są połączeniem notacji LIKE i zwykłej notacji wyrażeń regularnych (POSIX).

Składnia tego zapytania ma postać:

string SIMILAR TO wzorzec,
string NOT SIMILAR TO wzorzec

Oprócz funkcji zapożyczonych z LIKE, SIMILAR TO obsługuje te metaznaki pasujące do wzorca zapożyczone z wyrażeń regularnych POSIX:

- \| - oznacza naprzemienność (jedną z dwóch alternatyw).
- \* - oznacza powtórzenie poprzedniego elementu zero lub więcej razy.
- \+ - oznacza powtórzenie poprzedniego elementu jeden lub więcej razy.
- \? - oznacza powtórzenie poprzedniego elementu zero lub jeden raz.
- {m} - oznacza powtórzenie poprzedniego elementu dokładnie m razy.
- {m,} - oznacza powtórzenie poprzedniego elementu m lub więcej razy.
- {m, n} - oznacza powtórzenie poprzedniego elementu co najmniej mi nie więcej niż n razy.

Nawiasów () można używać do grupowania elementów w jeden element logiczny. Wyrażenie w nawiasie \[...\] określa klasę znaków, podobnie jak w wyrażeniach regularnych POSIX.

|Wyrażenie|Wynik|
|---|---:|
|'abc' SIMILAR TO 'abc'|     true|
|'abc' SIMILAR TO 'a'|        false|
|'abc' SIMILAR TO '%(b\|d)%'| true|
|'abc' SIMILAR TO '(b\|c)%'|   false|

## Składnia POSIX

Wyrażenia regularne POSIX zapewniają więcej reguł tworzenia wzorców niż operatory LIKE i SIMILAR TO. Wiele narzędzi uniksowych, takich jak grep, sed lub awk, używa języka dopasowywania wzorców podobnego do tego używanego w PostgreSQL.

Wyrażenie regularne to opis ciągu znaków przy użyciu symboli w celu utworzenia wzorca ciągu, która umożliwia dopasowanie wzorców. Mówi się, że łańcuch pasuje do wyrażenia regularnego, jeśli jest on członkiem zbioru regularnego opisanego przez wyrażenie regularne. Podobnie jak w przypadku LIKE, znaki wzorcowe dokładnie pasują do znaków łańcuchowych, chyba że są znakami specjalnymi w języku wyrażeń regularnych - ale wyrażenia regularne używają innych znaków specjalnych niż LIKE. W przeciwieństwie do wzorców LIKE, wyrażenie regularne może pasować w dowolnym miejscu ciągu, chyba że wyrażenie regularne jest wyraźnie zakotwiczone na początku (^) lub na końcu łańcucha ($).

Używanie tego rodzaju dopasowania ciągu znaków odbywa się przez operatory:

- ~	pasuje do wzorca , wielkość liter ma znaczenie
- ~\*	pasuje do wzorca , wielkość liter nie ma znaczenie	
- !~	nie pasuje do wzorca , wielkość liter ma znaczenie	
- !~\*	nie pasuje do wzorca , wielkość liter nie ma znaczenie

#### Przykład:
|Wyrażenie|Wynik|
|---|---:|
|'abc' ~ 'abc'|    true|
|'abc' ~ '^a' |    true|
|'abc' ~ '(b\|d)'|  true|
|'abc' ~ '^(b\|c)'| false|

Język zapytań regularnych w PostgreSQL który będzie omawiany na zajęciach składa się z:

|Wzorzec|Dopasowanie|
|---|---|
|\* |sekwencja 0 lub więcej dopasowań atomu|
|\+ |sekwencja 1 lub więcej dopasowań atomu|
|\? |sekwencja 0 lub 1 dopasowań atomu
|{m.} |sekwencja dokładnie m dopasowań atomu|
|{m,} |sekwencja m lub więcej dopasowań atomu|
|{m, n}| sekwencja od m do n (włącznie) dopasowań atomu; m nie może przekraczać n|
|^| dopasuj od początku łańcucha znaków|
|$| dopasuj od końca łańcucha znaków|

Formularze używające {...} są znane jako granice. Liczby min w granicach są liczbami całkowitymi dziesiętnymi bez znaku z dopuszczalnymi wartościami od 0 do 255 włącznie.

Pełny opis omawianych funkcjonalności dostępny jest w [dokumentacji PostgreSQL](https://www.postgresql.org/docs/9.3/functions-matching.html) 

## Wybrane funkcje działające na znakach

Poza funkcją dopasowania w PostgreSQL istnieje również szereg funkcji predefiniowanych niepozwalających działanie ciągach znaków. W poniższej tabeli przedstawiono wybrane funkcje:

|Funkcja| Opis| Przykład| Wynik|
|:---|:---|:---:|---:|
|ASCII| Zwraca wartość kodu ASCII znaku lub punktu kodu Unicode znaku UTF8<br><br>| ASCII ('A')| 65|
|CHR |Konwertuj kod ASCII na znak lub punkt kodu Unicode na znak UTF8<br><br>|CHR (65) |'A'|
|CONCAT| Połączenie dwóch lub więcej ciągów w jeden<br><br>| CONCAT('A', 'B', 'C')| 'ABC'|
|CONCAT_WS| Połączenie ciągów znaków z separatorem<br><br>| CONCAT_WS(',', 'A', 'B', 'C')| 'A, B, C'|
|FORMAT| Łącznie ciągów zgodnie z zadanym wzorcem formatowania<br><br>| FORMAT('Witaj% s', 'PostgreSQL') |'Witaj PostgreSQL'|
|INITCAP| Konwertuj łańcuch znaków w styl nagłówka<br><br>| INITCAP('CZEść wAM') |"Cześć Wam"|
|LEFT| Zwraca pierwszy n znaku z lewej strony ciągu<br><br>| LEFT('ABC', 1) |'A'|
|LENGTH| Zwraca liczbę znaków w ciągu<br><br>| LENGTH('ABC')| 3|
|LOWER| Konwertuj ciąg na małe litery<br><br>|LOWER ('czEŚĆ wAM')| 'cześć wam'|
|LPAD| Uzupełnieni z lewej strony ciągu do zadanej długości zadanym ciągiem<br><br>|LPAD('123', 5, '00')| '00123'|
|LTRIM| Usuwanie najdłuższego ciąg zawierającego określone znaki z lewej strony ciągu wejściowego<br><br>| LTRIM ('00123')|'123'|
|MD5| Zwraca skrót MD5 ciągu szesnastkowego<br><br>| MD5('ABC')||
|POSITION| Zwraca lokalizację pod łańcucha w ciągu<br><br>| POSTION('B' w 'A B C') |3|
|REGEXP_MATCHES| Dopasuj wyrażenie regularne POSIX do łańcucha i zwraca pasujące podciągi<br><br>| REGEXP_MATCHES ('ABC', '^(A)(..)$', 'g');| {ABC}|
|REGEXP_REPLACE| Zamienia podciągi pasujące do wyrażenia regularnego POSIX na nowy podciąg<br><br>| REGEXP_REPLACE ('John Doe','(.*)(.*)’,’\2, \1′);| 'Doe, John'|
|REPEAT| Powtarza ciąg określoną liczbę razy<br><br>| REPEAT('\*', 5)| '\*\*\*\*\*'|
|REPLACE| Zamienia wszystkie wystąpienia w ciągu pod łańcucha z podciągu na zadany <br><br>REPLACE('ABC', 'B', 'A')| 'AAC'|
|REVERSE| Odwrócenie ciągu<br><br>| REVERSE ('ABC') 'CBA'|
|RIGHT| Zwraca ostatnie n znaków w ciągu. Kiedy n jest ujemne, zwracaj wszystkie oprócz pierwszego<br><br>| RIGHT('ABC', 2)| 'BC'|
|RPAD| Uzupełnieni z prawej strony ciągu do zadanej długości zadanym ciągiem<br><br>| RPAD('ABC', 6, 'xo') | 'ABCxox'|
|RTRIM| Usuwa najdłuższy ciąg zawierający określone znaki z prawej strony ciągu wejściowego<br><br>| RTRIM 'abcxxzx', 'xyz')|'abc'|
|SPLIT_PART| Dzieli ciąg na określonym ograniczniku i zwraca n-ty pod łańcuch<br><br>| SPLIT_PART('2017-12-31′, ’-', 2)| ’12’|
|SUBSTRING| Wyodrębnia podciąg z ciągu<br><br>| SUBSTRING('ABC', 1,1)| 'A'|
|TRIM| Usuwa najdłuższy ciąg zawierający określone znaki z lewej, prawej lub obu ciągów wejściowych<br><br>| TRIM('ABC')| 'ABC'|
|UPPER| Konwertuje ciąg na wielkie litery<br><br>|UPPER('CZEść wAM') |'CZEŚĆ WAM'|


## Zadania:
1. Znajdź wszystkie nazwy krajów rozpoczynających się na P.
2. Znajdź wszystkie nazwy krajów rozpoczynających się  P i kończących na s.
3. Znajdź wszystkie tytuły filmów, w których znajdują się cyfry.
4. Znajdź wszystkich pracowników z podwójnym imieniem lub nazwiskiem.
5. Znajdź wszystkie nazwiska aktorów rozpoczynających się od P lub C i mających 5 znaków.
6. Znajdź wszystkie tytuły filmów, w których występują słowa Trip lub Alone.
7. Przeanalizuj zapytania:
	- select first_name from actor where first_name ~ '^ Al\[a: z, 1: 9\] *'
	- select first_name from actor where first_name ~ * '^ al\[a: z, 1: 9\] *'


In [26]:
import psycopg2 as pg
import pandas as pd
connection = pg.connect(host='pgsql-196447.vipserv.org', port=5432, dbname='wbauer_adb', user='wbauer_adb', password='adb2020');

1. Znajdź wszystkie nazwy krajów rozpoczynających się na P.

In [27]:
# Uzywam odpowiedniej tablicy country i szukam nazw w kolumnie country
# Korzystam z funkcji LIKE oraz składni 'P%', dzięki czemu otrzymuje wszystkie nazwy krajów zaczynających się na literę P
# % zastepuje dowolny ciag znakow
df = pd.read_sql("""SELECT country
                    FROM country
                    WHERE country LIKE 'P%'
                """,con=connection)
print('Kraje na literę "P":\n', df)

Kraje na literę "P":
        country
0     Pakistan
1     Paraguay
2         Peru
3  Philippines
4       Poland
5  Puerto Rico


2. Znajdź wszystkie nazwy krajów rozpoczynających się  P i kończących na s.

In [28]:
# To zadanie wykonuje w podobny sposób co zadanie 1, jednak uzywam skladni 'P%s', co pozwoli mi znalezc kraje rozpoczynajace sie na P i konczące na s.
# % pozwala na wypisanie znakow pomiedzy P a s.
df = pd.read_sql("""SELECT country
                    FROM country
                    WHERE country LIKE 'P%s' 
                """,con=connection)
print('Kraje na literę rozpoczynajace sie na "P" i kończące na "s":\n', df)

Kraje na literę rozpoczynajace sie na "P" i kończące na "s":
        country
0  Philippines


C:\Users\marci\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


3. Znajdź wszystkie tytuły filmów, w których znajdują się cyfry.

In [29]:
#CYFRY od 0 do 9
# ~~'[0-9]' ta składnia pozwala mi na znalezienie cyfr jako liczby w tytułach.
df = pd.read_sql("""SELECT title
                    FROM film
                    WHERE title ~~'[0-9]'
                """,con=connection)
print('Tytuły filmów posiadające cyfrę:\n', df)

#Szukam cyfr zadeklarowanych jako słowa
dx = pd.read_sql("""SELECT title
                    FROM film
                    WHERE title LIKE 'one' OR title LIKE 'two' OR title LIKE 'three' OR title LIKE 'four' OR title LIKE 'five' OR title LIKE 'six%' OR title LIKE 'Seven%' OR title LIKE 'eight' OR title LIKE 'nine' OR title LIKE 'zero'
                """,con=connection)
print('Tytuły filmów posiadające cyfrę:\n', dx)

Tytuły filmów posiadające cyfrę:
 Empty DataFrame
Columns: [title]
Index: []
Tytuły filmów posiadające cyfrę:
          title
0  Seven Swarm


C:\Users\marci\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\marci\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


4. Znajdź wszystkich pracowników z podwójnym imieniem lub nazwiskiem.

In [30]:
# Korzystajac z odpowiedniej tablicy oraz kolumn imie i nazwisko jestem w stanie znalezc szukane
# (first_name ~~ ' '), ta składnia pozwala na znalezienie powtarzajacych sie wartosci w kolumnie first_name
# analocziny zapis stosuje do kolumny last_name
df = pd.read_sql("""SELECT first_name, last_name
                    FROM staff
                    WHERE (first_name ~~ ' ') OR (last_name ~~ ' ')
                """,con=connection)
print('Pracownicy z podwójnym imieniem lub nazwiskiem:\n', df)

Pracownicy z podwójnym imieniem lub nazwiskiem:
 Empty DataFrame
Columns: [first_name, last_name]
Index: []


C:\Users\marci\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


5. Znajdź wszystkie nazwiska aktorów rozpoczynających się od P lub C i mających 5 znaków.

In [31]:
# Korzystam z tablicy actor oraz szukam po kolumnie last_name nazwiska aktorów 
# WHERE (last_name LIKE 'P%' OR last_name LIKE 'C%'), ta składnia pozwala na znalezienie nazwisk zaczynajacych sie na P lub C
# LENGTH(last_name) = 5, ta część umozliwia znalezienie tylko nazwisk o długości równej 5 znakom.
# ORDER BY last_name ASC, sortuje nazwiska alfabetycznie
df = pd.read_sql("""SELECT last_name
                    FROM actor
                    WHERE (last_name LIKE 'P%' OR last_name LIKE 'C%') AND LENGTH(last_name) = 5
                    ORDER BY last_name ASC
                """,con=connection)
print('Nazwiska aktorów rozpoczynające sie od "P" lub "C" i mających 5 znaków:\n', df)

Nazwiska aktorów rozpoczynające sie od "P" lub "C" i mających 5 znaków:
   last_name
0     Chase
1     Chase
2     Close
3     Crowe
4     Pesci
5     Posey


C:\Users\marci\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


6. Znajdź wszystkie tytuły filmów, w których występują słowa Trip lub Alone.

In [32]:
# Wyszukujac odpowiedniej tablicy oraz uzywajac funkcji LIKE oraz składni '%text%', jestem wstanie znalezc tytuły gdzie występuje szukany text. 
# Szukane słowo występuje na którym kolwiek miejscu ze względu na % który pobiera znaki przed(%text) jak i po(text%) szukanym słowie.
# text jest szukany na kazdej pozycji
df = pd.read_sql("""SELECT title
                    FROM film
                    WHERE title LIKE '%Trip%' OR title LIKE '%Alone%'
                """,con=connection)
print('Tytuły filmów posiadające słowa TRIP LUB ALONE:\n', df)

C:\Users\marci\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


Tytuły filmów posiadające słowa TRIP LUB ALONE:
            title
0     Alone Trip
1  Superfly Trip
2    Trip Newton
3   Varsity Trip


7. Przeanalizuj zapytania:

- select first_name from actor where first_name ~ '^ Al\[a: z, 1: 9\] *'

In [33]:
#To zadanie zaczynam od usunięcia spacji, które uniemożliwiają poprawne otwarcie programu.
# ~ pasuje do wzorca , wielkość liter ma znaczenie
# ^ dopasuj od początku łańcucha znaków
# ~'^Al*' - oznacza ze wyswietli sie kolumna z imionami aktorów zaczynajacych sie na 'Al' (duza litera ma znaczenie ze wzgledu na zadeklarowane ~), co istotne * odpowiada za dalszy ciąg znaków w wierszu.
# ~'^Al*[a]' - taka składnia pozwoli mi na wyświetlenie imienia które po Al posiada na następnym miejscu litere 'a'.
#  * bez tego znaku na koncu nie wyswietliło by się imię 'Al', ponieważ program czytałby jedynie polecenia zawarte w [] nie uwzględniając braku znaku po '^Al'
#PODSUMOWUJĄC: kod zwraca listę osób których imiona zaczynają się na Al i ich następne litery zawierają się w przedziale od a do z oraz cyfry są w zakresie od 1 do 9.
df = pd.read_sql("""SELECT first_name
                    FROM actor
                    WHERE first_name ~'^Al[a:z, 1:9]*'
                """,con=connection)
df

C:\Users\marci\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,first_name
0,Alec
1,Albert
2,Albert
3,Al
4,Alan


- select first_name from actor where first_name ~ * '^ al\[a: z, 1: 9\] *'

In [34]:
#usunałem nieistotne wstawione spacje które nie pozwalały włączyć programu.
# ~* pasuje do wzorca , wielkość liter nie ma znaczenie. Jest to glówna róźnica między przykładem wyżej, ponieważ ^al nie zostaje uznawany jako poczatek zmiennej str od małej litery. 
# Po usunięcu * nie wyszukamy imiona, ze wzgledu na to ze wszystkie zaczynają się z dużej litery
# ^ dopasuj od początku łańcucha znaków
#PODSUMOWUJAC: Dostajemy te same rezultaty co w przykładzie wyżej, jednak główną różnicą jest sposób zapisu, gdzie należy pamiętać o znaczeniu ~, jak i o ~*, które opisałem powyżej.
dx = pd.read_sql("""SELECT first_name 
                    FROM actor 
                    WHERE first_name ~*'^al[a:z, 1:9]*'
                """,con=connection)
dx

C:\Users\marci\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,first_name
0,Alec
1,Albert
2,Albert
3,Al
4,Alan


## Zadanie implementacyjne
Zaimplementuj wszystkie funkcje w pliku main.py zgodnie z opisem a następnie przetestuj je w notatniku.


In [35]:
import main

#main.film_title_case_insensitive(['BeD', 'BLOOD', 'SonS', 'son', 'cry'])
main.film_title_case_insensitive(['Giant', 'Harry', 'Birdcage', 'Iron'])
main.film_cast()
main.film_in_category()
main.film_in_category_case_insensitive()

UnboundLocalError: local variable 'd_film_title' referenced before assignment